<a href="https://colab.research.google.com/github/dariadementeva/Neighborhood_Quality_Working/blob/main/mapillary_streetscapes_lux_city.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installs the package.
!pip install mapillary

INFO: pip is looking at multiple versions of turfpy to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.2/978.2 kB 32.4 MB/s eta 0:00:00
  Created wheel for turfpy: filename=turfpy-0.0.7-py3-none-any.whl size=39094 sha256=34182c746c5c9b157cdefa605d4692c8d26f577d9a208edc0a514fc6489274bc
  Stored in directory: /root/.cache/pip/wheels/7b/4a/e1/faedb15d6c6c5ca5944a1ff705cdd6b20350781259bc2c437c
Successfully built turfpy
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the s

In [2]:
import mapillary.interface as mly
import requests
import json

In [3]:
token = 'MLY|25517726937837208|3f8625861dba933ed6b04631405be5bd'

In [4]:
mly.set_access_token(token)

{'token': 'SUCCESS'}

In [6]:
# Get image points close to Luxembourg
# The latitude of Luxembourg City, Luxembourg is 49.611622, and the longitude is 6.131935
data = mly.get_image_close_to(longitude=6.131935, latitude=49.611622).to_dict()

# Save the data as JSON
file_name = "luxembourg_city.json"
with open(file_name, mode="w") as f:
    json.dump(data, f, indent=4)

Requesting GET to https://tiles.mapillary.com/maps/vtp/mly1_public/2/14/8471/5583/?access_token=MLY%7C25517726937837208%7C3f8625861dba933ed6b04631405be5bd


INFO:mapillary.utils.client:Requesting GET to https://tiles.mapillary.com/maps/vtp/mly1_public/2/14/8471/5583/?access_token=MLY%7C25517726937837208%7C3f8625861dba933ed6b04631405be5bd


Response 200 OK received in 345ms


INFO:mapillary.utils.client:Response 200 OK received in 345ms


In [8]:
with open("luxembourg_city.json", "r") as f:
  data = json.load(f)

In [14]:
print(type(data)) # explore the structure
print(data.keys())
feature = data["features"][0]
print(feature.keys())
print(feature["properties"].keys())

<class 'dict'>
dict_keys(['type', 'features'])
dict_keys(['type', 'geometry', 'properties'])
dict_keys(['captured_at', 'compass_angle', 'creator_id', 'id', 'is_pano', 'sequence_id'])


In [15]:
print(len(data["features"])) # 8025 images
print(data["features"][0]["properties"]["id"])

8025
790758568474879


In [11]:
import pprint
pprint.pprint(data)

Streaming output truncated to the last 5000 lines.
                                            49.61899607107512],
                            'type': 'Point'},
               'properties': {'captured_at': 1524223883126,
                              'compass_angle': 343.89245605469,
                              'creator_id': 104293311811682,
                              'id': 195074978969815,
                              'is_pano': False,
                              'sequence_id': '4R4ZK2IdodP4NlyTDSxDYA'},
               'type': 'Feature'},
              {'geometry': {'coordinates': [6.133423447608948,
                                            49.61111663099837],
                            'type': 'Point'},
               'properties': {'captured_at': 1506518175516,
                              'compass_angle': 347.30780029297,
                              'creator_id': 100679362180421,
                              'id': 542019273449735,
                              'is_p

In [18]:
import json
import os
import time
import requests

# force image downdloading

ACCESS_TOKEN = "MLY|25517726937837208|3f8625861dba933ed6b04631405be5bd"
headers = {"Authorization": f"OAuth {ACCESS_TOKEN}"}

with open("luxembourg_city.json", "r") as f:
    data = json.load(f)

os.makedirs("images", exist_ok=True)

for i, feat in enumerate(data["features"], start=1):
    image_id = str(feat["properties"]["id"])

    # get urls
    meta = requests.get(
        f"https://graph.mapillary.com/{image_id}?fields=thumb_2048_url,thumb_1024_url",
        headers=headers,
        timeout=30
    )
    meta.raise_for_status()
    meta = meta.json()

    img_url = meta.get("thumb_2048_url") or meta.get("thumb_1024_url")
    if not img_url:
        print(f"[{i}] {image_id}: no thumbnail url")
        continue

    # download image
    img = requests.get(img_url, timeout=60)
    img.raise_for_status()

    out_path = os.path.join("images", f"{image_id}.jpg")
    with open(out_path, "wb") as out:
        out.write(img.content)

    if i % 100 == 0:
        print(f"Downloaded {i}/{len(data['features'])}")

    time.sleep(0.05)

Downloaded 100/8025
Downloaded 200/8025
Downloaded 300/8025
Downloaded 400/8025
Downloaded 500/8025
Downloaded 600/8025
Downloaded 700/8025
Downloaded 800/8025
Downloaded 900/8025
Downloaded 1000/8025
Downloaded 1100/8025
Downloaded 1200/8025
Downloaded 1300/8025
Downloaded 1400/8025
Downloaded 1500/8025
Downloaded 1600/8025
Downloaded 1700/8025
Downloaded 1800/8025
Downloaded 1900/8025
Downloaded 2000/8025
Downloaded 2100/8025
Downloaded 2200/8025
Downloaded 2300/8025
Downloaded 2400/8025
Downloaded 2500/8025
Downloaded 2600/8025
Downloaded 2700/8025
Downloaded 2800/8025
Downloaded 2900/8025
Downloaded 3000/8025
Downloaded 3100/8025
Downloaded 3200/8025
Downloaded 3300/8025
Downloaded 3400/8025
Downloaded 3500/8025
Downloaded 3600/8025
Downloaded 3700/8025
Downloaded 3800/8025
Downloaded 3900/8025
Downloaded 4000/8025
Downloaded 4100/8025
Downloaded 4200/8025
Downloaded 4300/8025
Downloaded 4400/8025
Downloaded 4500/8025
Downloaded 4600/8025
Downloaded 4700/8025
Downloaded 4800/8025
D

In [20]:

# calculate missing images
import os, json

saved = set(f.split(".")[0] for f in os.listdir("/content/images"))

with open("luxembourg_city.json", "r") as f:
    data = json.load(f)

missing = [
    feat["properties"]["id"]
    for feat in data["features"]
    if str(feat["properties"]["id"]) not in saved
]

print("Missing:", len(missing))
missing[:10]

Missing: 0


[]

In [21]:
# save as zip

!zip -r luxembourg_images.zip /content/images
from google.colab import files
files.download("luxembourg_images.zip")

Streaming output truncated to the last 5000 lines.
  adding: content/images/206945295098253.jpg (deflated 0%)
  adding: content/images/170652408394541.jpg (deflated 0%)
  adding: content/images/922541974974122.jpg (deflated 1%)
  adding: content/images/3903090846449739.jpg (deflated 0%)
  adding: content/images/195359435613743.jpg (deflated 0%)
  adding: content/images/840772121094032.jpg (deflated 0%)
  adding: content/images/768556715238866.jpg (deflated 0%)
  adding: content/images/799071341029624.jpg (deflated 0%)
  adding: content/images/312984923676447.jpg (deflated 0%)
  adding: content/images/381603929733288.jpg (deflated 1%)
  adding: content/images/923358345301554.jpg (deflated 0%)
  adding: content/images/1197638300955338.jpg (deflated 1%)
  adding: content/images/2578840962409717.jpg (deflated 1%)
  adding: content/images/3741799219423257.jpg (deflated 1%)
  adding: content/images/1522181978673715.jpg (deflated 0%)
  adding: content/images/800194940882020.jpg (deflated 0%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>